In [1]:
# cartpole 버티기
# Q-learning : (off-policy TD control) 방식으로 카트폴 환경을 학습하기
# MDP(Markov Decision Process) 기반의 강화학습 알고리즘을 사용
# MDP의 5가지 환경 구성요소
# S (State ): 에이전트가 놓일 수 있는 모든 가능한 상황
# A (Action): 에이전트가 각 상태에서 할 수 있는 모든 가능한 동작
# R (Reward)): 에이전트가 상태 s에서 행동 a를 취해 새로운 상태 $s'$에 도달했을 때 받는 보상
# P (Transition Probability): 에이전트가 상태 s에서 행동 a를 취했을 때, 다음 상태 $s'$로 이동할 확률
# π(Policy)): 어떤 상태에서 어떤 행동을 할지를 결정

!pip install gymnasium[classic-control]




In [2]:
import gymnasium as gym
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from matplotlib.animation import FuncAnimation
from IPython.display import HTML


In [3]:
from logging import raiseExceptions
# 환경 설정
env = gym.make('CartPole-v1') # 카트에 막대 (pole)를 수직으로 세운 채 좌우로 움직여 균형을 유지하는 환경을 제공
print(env.observation_space) # 관측 공간
# 카트 위치 , 카트 속도 , 막대 기울기, 막대 각 속도
obs_space_low = np.array([-2.4, -3.0, -0.5, -2.0])
obs_space_high = np.array([2.4, 3.0, 0.5, 2.0])

# 상태 공간 이산화 수준 설정. Q-table은 연속적인 상태를 다를 수 없으므로 구간으로 나눠줘야함
state_bins = [6, 12, 6, 12] # 카트폴의 4가지 관측값에 대해 4가지 이산화 수준을 순서대로 적용해야 하므로, 순서가 보장되는 리스트나 튜플을 사용

q_table = np.zeros(state_bins + [env.action_space.n])
print(q_table.shape)

# 상태 이산화 처리 함수
def discretize_state(state):
    ratios = (state - obs_space_low) / (obs_space_high - obs_space_low)
    # ex) (0-(-2.4))/(2.4--(-2.4)) =
    print('ratios :' , ratios)
    discrete = (ratios * state_bins).astype(int) # 구간이 선택됨
    print('discrete :' , discrete)

    return tuple(np.clip(discrete, 0, np.array(state_bins) -1 )) # 원본 배열, 허용 최소값, 허용 최대값

# discretize_state 결과실험
"""
ex_state = np.array([1.0, 0.5, 0.1, -1.0])
dis_index = discretize_state(ex_state)
print('Q-table 인덱스 :', dis_index)
"""
# Q-learning의 하이퍼파라메터 설정
alpha = 0.1 # Q-테이블 값을 얼마나 빠르게 업데이트할지 결정. 값이 클수록 최근 학습 결과에 더 큰 비중
gammer = 0.99 # 미래의 보상을 얼마나 중요하게 생각할지 결정
epsilon = 1.0 #초기에 1.0으로 설정하여 에이전트가 무작위로 행동하며 탐험을 많이 하도록
epsilon_decay =0.99   # 에피소드가 진행될수록 엡실론 값을 줄여 탐험을 줄이고 학습된 행동을 활용
esilon_min =0.05      # 엡실론이 너무 작아지지 않도록 최솟값을 설정
episodes = 1000
reward_list=[]  # 각 에피소드에서 받은 총 보상
trajector = []  # 궤적 위치 저장 - 애니메이션 시각화용
best_reward = 0 # 최고의 총 보상 저장

Box([-4.8               -inf -0.41887903        -inf], [4.8               inf 0.41887903        inf], (4,), float32)
(6, 12, 6, 12, 2)
